In [1]:
import time
import numpy as np
from tqdm import tqdm_notebook
import datasets
from tqdm import tqdm

import json
import openai
import os

from utils.eval_utils import micro_precision, micro_recall, f1_score
from utils.openai_utils import LLMTripletExtractor
from utils.index_utils import Aligner
from utils.verifier_utils import TripletFilter

import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

df = pd.read_csv('assessment_100_samples_from_test_small.csv', index_col=0)
df

,ID,Text,Entities,Natural language text,SyntIE Generated triplets,gpt Generated triplets
0,12534,Bob Dylan is an American singer-songwriter who...,Bob_Dylan Bob_Dylan_discography,Bob Dylan is an American singer-songwriter. Bl...,Blonde_on_Blonde performer Bob_Dylan\nBlonde_o...,Bob Dylan occupation Singer-songwriter\nBlonde...
1,1926,Amalie Murtfeldt passed away in Bremen.,Amalie_Murtfeldt Bremen,"On June 28, 1888, aged 60, Amalie, who had bee...",Funeral manifestation of Death\nFuneral descri...,NaN
2,18858,"Gibson, Georgia is the capital of Glascock Cou...","Gibson,_Georgia Glascock_County,_Georgia Easte...","Gibson is a city in Glascock County, Georgia, ...","Gibson,_Georgia country United_States\nGibson,...",Glascock County Courthouse instance of Nationa...
3,44871,Parsonsfield Seminary is located in Parsonsfie...,"Parsonsfield_Seminary Parsonsfield,_Maine Unit...","Parsonsfield Seminary, which operated from 183...",Parsonsfield_Seminary country United_States\nP...,Parsonsfield Seminary instance of Educational ...
4,5573,Hatfield Forest is operated by the National Tr...,Hatfield_Forest National_Trust Site_of_Special...,Hatfield Forest is a biological Site of Specia...,Hatfield_Forest significant place Bishop's_Sto...,Hatfield Forest instance of Site of Special Sc...
...,...,...,...,...,...,...
95,26682,Åsensbruk is located in the UTC+01:00 time zone.,Åsensbruk UTC+01:00,Åsensbruk is a locality situated in Mellerud M...,Mellerud_Municipality legal form Municipalitie...,Åsensbruk located in the administrative territ...
96,43737,Richard Brookhiser is a historian who received...,Richard_Brookhiser National_Humanities_Medal R...,"In 2008, President George W. Bush awarded Broo...",George_W._Bush award received National_Humanit...,National Humanities Medal instance of Medal\nW...
97,45299,The Pentium III processor was built on the P6 ...,Pentium_III Socket_370 P6_(microarchitecture) ...,The Pentium III brand refers to Intel's 32-bit...,Pentium_III brand Pentium\nPentium_III microar...,Pentium III manufacturer Intel\nPentium III in...
98,1424,Mel Bush made her first appearance in the Doct...,Mel_Bush Terror_of_the_Vervoids Sixth_Doctor S...,"Melanie ""Mel"" Bush is a fictional character pl...",Mel_Bush sidekick of Sixth_Doctor\nMel_Bush pr...,NaN


In [3]:
texts = list(df['Natural language text'])
texts[:10]

['Bob Dylan is an American singer-songwriter. Blonde on Blonde is his seventh studio album.',
 'On June 28, 1888, aged 60, Amalie, who had been weakening, died. Her grave is located in the Riensberg Cemetery.',
 'Gibson is a city in Glascock County, Georgia, United States. The city is the county seat of Glascock County and home to the Glascock County Courthouse, a National Register of Historic Places listed site.',
 'Parsonsfield Seminary, which operated from 1832 to 1949, was a well-known Free Will Baptist school in North Parsonsfield, Maine, in the United States. ',
 "Hatfield Forest is a biological Site of Special Scientific Interest in Essex, three miles east of Bishop's Stortford. It is also a National Nature Reserve and a Nature Conservation Review site. It is owned and managed by the National Trust.",
 'The Sakuradamon Incident was the assassination of Ii Naosuke, Chief Minister of the Tokugawa shogunate, on March 24, 1860 by rōnin samurai of the Mito Domain and Satsuma Domain.'

## Auxilary

In [4]:
def parse_output(output):
    try:
        output = re.findall("\[([\s\S]*?)\]", output)
        return "[" + output[0] + "]"

    except Exception as e:
        print(e, output)

############## transform target triplets into tuples with labels and ids to calculate metrics properly ############## 
def transform_targets_syntie(sample):
    
    targets = {"triplet_texts": [], "triplet_ids": []}
    for t in sample:
        rel = eval(t['predicate'])
        rel_label = rel['surfaceform'].strip()
        rel_id = rel['uri']
        
        head = eval(t['subject'])
        head_label = " ".join(head['surfaceform'].split("_")).strip()
        head_id = head['uri']

        tail =  eval(t['object'])
        tail_label = " ".join(tail['surfaceform'].split("_")).strip()
        tail_id = tail['uri']

        targets['triplet_texts'].append((head_label, rel_label, tail_label))
        targets['triplet_ids'].append((head_id, rel_id, tail_id))

    return targets


############## align extracted triplets to wikidata ids ############## 
def align_outputs(outputs, aligner, triplet_filter, verify=True):
    transformed_output_ids = []
    filtered_triplets = []

    outputs = json.loads(outputs)

    for res in outputs:
        rel = res['relation']
        rel = " ".join(rel.split("_")).strip()

        head, tail = res['subject'], res['object']
        head = " ".join(head.split("_")).strip()
        tail = " ".join(tail.split("_")).strip()

        if (rel in aligner.relation2id) and (head in aligner.entity2id) and (tail in aligner.entity2id):
            rel_id = aligner.relation2id[rel]
            head_id = aligner.entity2id[head]
            tail_id = aligner.entity2id[tail]

            if verify:

                if triplet_filter.check_triplet_validity(head_id, rel_id, tail_id):
                    transformed_output_ids.append((head_id, rel_id, tail_id))

                elif triplet_filter.check_triplet_validity(tail_id, rel_id, head_id):
                    transformed_output_ids.append((tail_id, rel_id, head_id))
                
                else:
                    filtered_triplets.append({"subject": head, "relation": rel, "object": tail})

            else:
                transformed_output_ids.append((head_id, rel_id, tail_id))
                
        else:
            filtered_triplets.append({"subject": head, "relation": rel, "object": tail})
        
    return {'transformed_outputs': transformed_output_ids, 'filtered_triplets': filtered_triplets}

## Inference

In [5]:
aligner = Aligner(device='cuda:5')
triplet_filter = TripletFilter()
extractor = LLMTripletExtractor()

In [6]:
generated_triplets = []

for i, text in tqdm(enumerate(texts), total=len(texts)):

    ############## first step prompting ##############
    extracted_triplets = extractor.get_completion_first_query(text)
    extracted_triplets = parse_output(extracted_triplets)

    ############## second step aligning all entity and relation names ##############

    similar_relations = aligner.top_relations_by_llm_output(llm_output=extracted_triplets)
    similar_entities = aligner.top_entities_by_llm_output(llm_output=extracted_triplets, entity_type='subject')
    similar_entities.update(aligner.top_entities_by_llm_output(llm_output=extracted_triplets, entity_type='object'))


    ############## second step prompting for choosing among aligned entity and relation names ##############
    second_step_output = extractor.get_completion_second_query(similar_entities=similar_entities, 
        similar_relations=similar_relations, text=text, triplets=extracted_triplets)
    
    second_step_output = parse_output(second_step_output)

    ############## second step entity linking, with constraints verification ##############

    second_step_output_ids = align_outputs(second_step_output, aligner=aligner, triplet_filter=triplet_filter, verify=True)['transformed_outputs']

    generated_triplets.append([(aligner.id2entity[item[0]], aligner.id2relation[item[1]], aligner.id2entity[item[2]]) \
        for item in second_step_output_ids])



  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [24:03<00:00, 14.43s/it]


In [8]:
generated_triplets

[[('Bob Dylan', 'occupation', 'Singer-songwriter'),
  ('Blonde on Blonde', 'instance of', 'Album Album'),
  ('Blonde on Blonde', 'performer', 'Bob Dylan')],
 [],
 [('Glascock County Courthouse',
   'instance of',
   'National Historic Site (United States)')],
 [('Parsonsfield Seminary', 'instance of', 'Educational institution'),
  ('Parsonsfield Seminary', 'country', 'United States'),
  ('Parsonsfield Seminary', 'affiliation', 'Free Will Baptist')],
 [('Hatfield Forest', 'instance of', 'Site of Special Scientific Interest'),
  ('Hatfield Forest',
   'located in the administrative territorial entity',
   'Essex'),
  ('Hatfield Forest', 'direction', 'East of West'),
  ('Hatfield Forest', 'owned by', 'National Trust'),
  ('Hatfield Forest', 'instance of', 'A Nature Conservation Review')],
 [('Ii Naosuke', 'instance of', 'Human')],
 [],
 [('Princess Thyra of Denmark', 'country of citizenship', 'Denmark'),
  ('Princess Thyra of Denmark', 'child', 'Christian IX of Denmark'),
  ('Princess Thy

In [9]:
outputs = []
for triplets in generated_triplets:
    output = []
    for triplet in triplets:
        output.append(" ".join(triplet))
    outputs.append('\n'.join(output))

In [10]:
outputs

['Bob Dylan occupation Singer-songwriter\nBlonde on Blonde instance of Album Album\nBlonde on Blonde performer Bob Dylan',
 '',
 'Glascock County Courthouse instance of National Historic Site (United States)',
 'Parsonsfield Seminary instance of Educational institution\nParsonsfield Seminary country United States\nParsonsfield Seminary affiliation Free Will Baptist',
 'Hatfield Forest instance of Site of Special Scientific Interest\nHatfield Forest located in the administrative territorial entity Essex\nHatfield Forest direction East of West\nHatfield Forest owned by National Trust\nHatfield Forest instance of A Nature Conservation Review',
 'Ii Naosuke instance of Human',
 '',
 'Princess Thyra of Denmark country of citizenship Denmark\nPrincess Thyra of Denmark child Christian IX of Denmark\nPrincess Thyra of Denmark child Louise of Hesse-Kassel',
 '',
 '',
 'Rantalinna Manor instance of Manor house\nRantalinna Manor located in the administrative territorial entity Ruokolahti\nRantali

In [11]:
df['gpt Generated triplets'] = outputs

In [12]:
df

,ID,Text,Entities,Natural language text,SyntIE Generated triplets,gpt Generated triplets
0,12534,Bob Dylan is an American singer-songwriter who...,Bob_Dylan Bob_Dylan_discography,Bob Dylan is an American singer-songwriter. Bl...,Blonde_on_Blonde performer Bob_Dylan\nBlonde_o...,Bob Dylan occupation Singer-songwriter\nBlonde...
1,1926,Amalie Murtfeldt passed away in Bremen.,Amalie_Murtfeldt Bremen,"On June 28, 1888, aged 60, Amalie, who had bee...",Funeral manifestation of Death\nFuneral descri...,
2,18858,"Gibson, Georgia is the capital of Glascock Cou...","Gibson,_Georgia Glascock_County,_Georgia Easte...","Gibson is a city in Glascock County, Georgia, ...","Gibson,_Georgia country United_States\nGibson,...",Glascock County Courthouse instance of Nationa...
3,44871,Parsonsfield Seminary is located in Parsonsfie...,"Parsonsfield_Seminary Parsonsfield,_Maine Unit...","Parsonsfield Seminary, which operated from 183...",Parsonsfield_Seminary country United_States\nP...,Parsonsfield Seminary instance of Educational ...
4,5573,Hatfield Forest is operated by the National Tr...,Hatfield_Forest National_Trust Site_of_Special...,Hatfield Forest is a biological Site of Specia...,Hatfield_Forest significant place Bishop's_Sto...,Hatfield Forest instance of Site of Special Sc...
...,...,...,...,...,...,...
95,26682,Åsensbruk is located in the UTC+01:00 time zone.,Åsensbruk UTC+01:00,Åsensbruk is a locality situated in Mellerud M...,Mellerud_Municipality legal form Municipalitie...,Åsensbruk located in the administrative territ...
96,43737,Richard Brookhiser is a historian who received...,Richard_Brookhiser National_Humanities_Medal R...,"In 2008, President George W. Bush awarded Broo...",George_W._Bush award received National_Humanit...,National Humanities Medal instance of Medal\nW...
97,45299,The Pentium III processor was built on the P6 ...,Pentium_III Socket_370 P6_(microarchitecture) ...,The Pentium III brand refers to Intel's 32-bit...,Pentium_III brand Pentium\nPentium_III microar...,Pentium III manufacturer Intel\nPentium III in...
98,1424,Mel Bush made her first appearance in the Doct...,Mel_Bush Terror_of_the_Vervoids Sixth_Doctor S...,"Melanie ""Mel"" Bush is a fictional character pl...",Mel_Bush sidekick of Sixth_Doctor\nMel_Bush pr...,


In [13]:
df.to_csv('assessment_100_samples_from_test_small.csv')

## Evaluation on SynthIE

In [160]:
import datasets

dataset_name = 'synthie_code' # 'synthie_code', 'rebel', 'synthie_text_pc', 'synthie_code_pc', 'rebel_pc'
split = "train" # "train", "test", "test_small"

dataset = datasets.load_dataset(f"martinjosifoski/SynthIE", dataset_name, split=split)

In [183]:
dataset[156]

{'id': 156,
 'text': 'Dr. Babasaheb Ambedkar College of Arts, Commerce and Science is an academic institution in India.',
 'triplets': [{'subject': "{'surfaceform': 'Dr._Babasaheb_Ambedkar_College_of_Arts,_Commerce_and_Science', 'uri': 'Q48726547'}",
   'predicate': "{'surfaceform': 'instance of', 'uri': 'P31'}",
   'object': "{'surfaceform': 'Academic_institution', 'uri': 'Q4671277'}"},
  {'subject': "{'surfaceform': 'Dr._Babasaheb_Ambedkar_College_of_Arts,_Commerce_and_Science', 'uri': 'Q48726547'}",
   'predicate': "{'surfaceform': 'country', 'uri': 'P17'}",
   'object': "{'surfaceform': 'India', 'uri': 'Q668'}"},
  {'subject': "{'surfaceform': 'Academic_institution', 'uri': 'Q4671277'}",
   'predicate': "{'surfaceform': 'subclass of', 'uri': 'P279'}",
   'object': "{'surfaceform': 'Educational_institution', 'uri': 'Q2385804'}"},
  {'subject': "{'surfaceform': 'Academic_institution', 'uri': 'Q4671277'}",
   'predicate': "{'surfaceform': 'subclass of', 'uri': 'P279'}",
   'object': "

In [157]:
len(train_entities), len(test_entities)

(26828, 27843)

In [159]:
len(set(train_entities) & set(test_entities))

5270

In [ ]:
    dataset[0]['entities']

In [147]:
for i in range(0, len(dataset)):
    if dataset[i]['id'] == 1925:
        print(dataset[i])

{'id': 1925, 'text': 'Bach flower remedies are a form of alternative medicine named after Edward Bach, and they are related to both flowers and plants. Flowers are part of a shoot.', 'triplets': [{'subject': "{'surfaceform': 'Bach_flower_remedies', 'uri': 'Q788110'}", 'predicate': "{'surfaceform': 'relative to', 'uri': 'P2210'}", 'object': "{'surfaceform': 'Flower', 'uri': 'Q506'}"}, {'subject': "{'surfaceform': 'Bach_flower_remedies', 'uri': 'Q788110'}", 'predicate': "{'surfaceform': 'relative to', 'uri': 'P2210'}", 'object': "{'surfaceform': 'Plant', 'uri': 'Q756'}"}, {'subject': "{'surfaceform': 'Bach_flower_remedies', 'uri': 'Q788110'}", 'predicate': "{'surfaceform': 'relative to', 'uri': 'P2210'}", 'object': "{'surfaceform': 'Alternative_medicine', 'uri': 'Q188504'}"}, {'subject': "{'surfaceform': 'Bach_flower_remedies', 'uri': 'Q788110'}", 'predicate': "{'surfaceform': 'named after', 'uri': 'P138'}", 'object': "{'surfaceform': 'Edward_Bach', 'uri': 'Q969176'}"}, {'subject': "{'su

In [7]:
gpt_res = []

for i in range(0, 3):

    logs_filename = f'logs/logs_improved_prompt_synthie_text_test_small_10000_launch_{int(i)}.jsonl'
    gpt_res.append({})

    with open(logs_filename, 'r') as f:
        for line in f:
            out = json.loads(line)[-1]
            index = dataset[out['index']]['id']
            gpt_res[i][index] = out
            

In [40]:
zero_shot_gpt_res = {}

for i in range(0, 2):

    logs_filename = f'logs/logs_improved_prompt_synthie_text_test_small_10000_launch_{int(i+4)}.jsonl'

    with open(logs_filename, 'r') as f:
        for line in f:
            out = json.loads(line)[-1]
            index = dataset[out['index']]['id']
            zero_shot_gpt_res[index] = out
            

In [41]:
len(gpt_res[0].keys()), len(gpt_res[1].keys()), len(gpt_res[2].keys())

(5847, 6302, 5596)

In [42]:
len(zero_shot_gpt_res)

5587

In [12]:
logs_synthie_filename_1 = '/home/chepurova/SynthIE/notebooks/logs/logs_0_to_5000.jsonl'
synthie_res = {}

with open(logs_synthie_filename_1, 'r') as f:
    for line in f:
        out = json.loads(line)
        index = out['id']
        synthie_res[index] = out

logs_synthie_filename_2 = '/home/chepurova/SynthIE/notebooks/logs/logs_5000_to_10000.jsonl'
with open(logs_synthie_filename_2, 'r') as f:
    for line in f:
        out = json.loads(line)
        index = out['id']
        synthie_res[index] = out



In [13]:
len(synthie_res)

10000

In [14]:
gpt_res[1].keys()

dict_keys([856, 43678, 27842, 44904, 34729, 7599, 19328, 48010, 39054, 8755, 36270, 6173, 22219, 40183, 22122, 25622, 7377, 6538, 8414, 41401, 4463, 42017, 9351, 35521, 9270, 16035, 40904, 17315, 48815, 32999, 34508, 13713, 26109, 3440, 12129, 26755, 35805, 29751, 17986, 3097, 36902, 38043, 2888, 15656, 8407, 46285, 37473, 41487, 38641, 23591, 15886, 16110, 27616, 34870, 36731, 34347, 12899, 16827, 10022, 26504, 26008, 33121, 5272, 42947, 27173, 3902, 30145, 34249, 17251, 20353, 25706, 8578, 41841, 11448, 2194, 2594, 24374, 39151, 22733, 6432, 35141, 48336, 8099, 49172, 28488, 36614, 24861, 21302, 47004, 17060, 9091, 2838, 36879, 26145, 299, 37973, 27470, 34879, 10886, 15109, 42785, 23551, 38427, 42439, 50103, 9238, 33628, 28352, 10504, 33141, 44088, 6336, 29983, 22793, 29370, 1829, 35622, 43266, 111, 33122, 35198, 19268, 13997, 4911, 35820, 22957, 30021, 44746, 18786, 43026, 21406, 11113, 45056, 39074, 39973, 50188, 1558, 30537, 7042, 45193, 27395, 37496, 38156, 11473, 46206, 19083, 1

In [15]:
synthie_res.keys()

dict_keys([856, 43678, 27842, 44904, 34729, 7599, 19328, 48010, 39054, 8755, 36270, 6173, 22219, 40183, 22122, 25622, 7377, 6538, 8414, 41401, 4463, 42017, 9351, 35521, 9270, 16035, 40904, 17315, 48815, 32999, 34508, 13713, 26109, 3440, 12129, 26755, 35805, 29751, 17986, 3097, 36902, 38043, 2888, 15656, 8407, 46285, 37473, 41487, 38641, 23591, 15886, 16110, 27616, 34870, 36731, 34347, 12899, 16827, 10022, 26504, 26008, 33121, 5272, 42947, 27173, 3902, 30145, 34249, 17251, 20353, 25706, 8578, 41841, 11448, 2194, 2594, 24374, 39151, 22733, 6432, 35141, 48336, 8099, 49172, 28488, 36614, 24861, 21302, 47004, 17060, 9091, 2838, 36879, 26145, 299, 37973, 27470, 34879, 10886, 15109, 42785, 23551, 38427, 42439, 50103, 9238, 33628, 28352, 10504, 33141, 44088, 6336, 29983, 22793, 29370, 1829, 35622, 43266, 111, 33122, 35198, 19268, 13997, 4911, 35820, 22957, 30021, 44746, 18786, 43026, 21406, 11113, 45056, 39074, 39973, 50188, 1558, 30537, 7042, 45193, 27395, 37496, 38156, 11473, 46206, 19083, 1

In [48]:
eval_indices = list(set(synthie_res.keys()) & set(gpt_res[0].keys()) & set(gpt_res[1].keys()) & set(gpt_res[2].keys()) & set(zero_shot_gpt_res.keys()))
len(eval_indices)

5587

In [49]:
import numpy as np
np.random.seed(seed=42)
eval_ids = np.random.choice(eval_indices, size=5000, replace=False)
len(eval_ids)

5000

In [50]:
def convert(s):
    return "_".join(s.split())

In [82]:
target_texts = []
first_step_texts = {i: [] for i in range(3)}
first_step_unverified_texts = {i: [] for i in range(3)}
second_step_texts = {i: [] for i in range(3)}
second_step_unverified_texts = {i: [] for i in range(3)}
zero_shot_second_step_texts = []
synthie_texts = []
texts = []

for i in eval_ids:
    texts.append(synthie_res[i]['text'])
    synthie_texts.append(list(map(tuple,synthie_res[i]['output_texts'])))
    target_texts.append(list(map(tuple, synthie_res[i]['target_texts'])))

    zero_shot_second_step_texts.append(list(map(lambda x: (convert(x[0]), x[1], convert(x[2])), zero_shot_gpt_res[i]['second_step_output'])))

    for num, res in enumerate(gpt_res): 
        assert synthie_res[i]['text'] == res[i]['text']

        first_step_texts[num].append(list(map(lambda x: (convert(x[0]), x[1], convert(x[2])), res[i]['first_step_output'])))
        first_step_unverified_texts[num].append(list(map(lambda x: (convert(x[0]), x[1], convert(x[2])), res[i]['first_step_unverified_output'])))

        second_step_texts[num].append(list(map(lambda x: (convert(x[0]), x[1], convert(x[2])), res[i]['second_step_output'])))
        second_step_unverified_texts[num].append(list(map(lambda x: (convert(x[0]), x[1], convert(x[2])), res[i]['second_step_unverified_output'])))


In [83]:
len(synthie_texts), len(first_step_unverified_texts[0]), len(first_step_unverified_texts[1]), len(first_step_unverified_texts[2])

(5000, 5000, 5000, 5000)

In [53]:
precision = micro_precision(synthie_texts, target_texts)
recall = micro_recall(synthie_texts, target_texts)
f1 = f1_score(precision, recall)
precision, recall, f1

(0.8764598861435531, 0.8845060412224591, 0.8804645815523392)

In [74]:
zero_shot_precision = micro_precision(zero_shot_second_step_texts, target_texts)
zero_shot_recall = micro_recall(zero_shot_second_step_texts, target_texts)
zero_shot_f1 = f1_score(zero_shot_precision, zero_shot_recall)
zero_shot_precision, zero_shot_recall, zero_shot_f1

(0.6572237960339944, 0.09618573797678276, 0.1678119349005425)

In [77]:
target_texts[10]

[('Budget_Direct', 'industry', 'Insurance'),
 ('Budget_Direct', 'legal form', 'Company'),
 ('Budget_Direct', 'headquarters location', 'Brisbane')]

In [54]:
first_step_precision = []
first_step_recall = []
first_step_f1 = []

for i in range(0, 3):
    first_step_precision.append(micro_precision(first_step_texts[i], target_texts))
    first_step_recall.append(micro_recall(first_step_texts[i], target_texts))
    first_step_f1.append(f1_score(first_step_precision[i], first_step_recall[i]))
first_step_precision, first_step_recall, first_step_f1

([0.6099285017874553, 0.6440329218106996, 0.5598185452867908],
 [0.4446221274579484, 0.4449182658137882, 0.4312366737739872],
 [0.5143189915045219, 0.5262715426649852, 0.487186349949816])

In [55]:
np.mean(first_step_precision), np.std(first_step_precision)

(0.6045933229616486, 0.0345867354454004)

In [56]:
np.mean(first_step_recall), np.std(first_step_recall)

(0.44025902234857456, 0.00638090927859392)

In [57]:
np.mean(first_step_f1), np.std(first_step_f1)

(0.5092589613731078, 0.016352696028353514)

In [58]:
first_step_unverified_precision = []
first_step_unverified_recall = []
first_step_unverified_f1 = []

for i in range(0, 3):
    first_step_unverified_precision.append(micro_precision(first_step_unverified_texts[i], target_texts))
    first_step_unverified_recall.append(micro_recall(first_step_unverified_texts[i], target_texts))
    first_step_unverified_f1.append(f1_score(first_step_unverified_precision[i], first_step_unverified_recall[i]))

first_step_unverified_precision, first_step_unverified_recall, first_step_unverified_f1

([0.5890700956562647, 0.6210222588623248, 0.5390486069946651],
 [0.4449774934849562, 0.44616204690831557, 0.4308813077469794],
 [0.506984276941764, 0.5192665609705659, 0.4789335088874259])

In [59]:
np.mean(first_step_unverified_precision), np.std(first_step_unverified_precision)

(0.5830469871710848, 0.033735523087671755)

In [60]:
np.mean(first_step_unverified_recall), np.std(first_step_unverified_recall)

(0.44067361604675037, 0.006941074272354905)

In [61]:
np.mean(first_step_unverified_f1), np.std(first_step_unverified_f1)

(0.5017281155999186, 0.016880150011654767)

In [65]:
second_step_precision = []
second_step_recall = []
second_step_f1 = []

for i in range(0, 3):
    second_step_precision.append(micro_precision(second_step_texts[i], target_texts))
    second_step_recall.append(micro_recall(second_step_texts[i], target_texts))
    second_step_f1.append(f1_score(second_step_precision[i], second_step_recall[i]))

second_step_precision, second_step_recall, second_step_f1

([0.591933731667572, 0.6314592366195181, 0.5532320600012712],
 [0.5163468372423596, 0.5261786306562426, 0.515517649846008],
 [0.5515626977097304, 0.5740315962911511, 0.5337094153355613])

In [66]:
np.mean(second_step_precision), np.std(second_step_precision)

(0.5922083427627871, 0.031936701445785)

In [67]:
np.mean(second_step_recall), np.std(second_step_recall)

(0.51934770591487, 0.004842040723447672)

In [68]:
np.mean(second_step_f1), np.std(second_step_f1)

(0.5531012364454809, 0.016497371413549212)

In [69]:
second_step_unverified_precision = []
second_step_unverified_recall = []
second_step_unverified_f1 = []

for i in range(0, 3):
    second_step_unverified_precision.append(micro_precision(second_step_unverified_texts[i], target_texts))
    second_step_unverified_recall.append(micro_recall(second_step_unverified_texts[i], target_texts))
    second_step_unverified_f1.append(f1_score(second_step_unverified_precision[i], second_step_unverified_recall[i]))

second_step_unverified_precision, second_step_unverified_recall, second_step_unverified_f1

([0.5538823230397764, 0.5886440341473099, 0.5151119292203551],
 [0.5162876095711917, 0.5268301350390903, 0.5138000473821369],
 [0.5344246214211269, 0.5560243788091889, 0.514455151964418])

In [70]:
np.mean(second_step_unverified_precision), np.std(second_step_unverified_precision)

(0.5525460954691471, 0.03003422205681996)

In [71]:
np.mean(second_step_unverified_recall), np.std(second_step_unverified_recall)

(0.5189725973308063, 0.005648165810770323)

In [72]:
np.mean(second_step_unverified_f1), np.std(second_step_unverified_f1)

(0.5349680507315779, 0.016974915643965104)